## data

In [11]:
import json
with open('Data_1910.json', 'r', encoding='utf-8') as f:
  json_dict = json.loads(f.read())

In [12]:
len(json_dict)

1857

In [38]:
def rename(title, index):
  number = title.split()[1]
  number_change = number[:-1] + f"_{index}."
  title = title.replace(number, number_change)
  return title

def split_nested_dict_by_length(dictionary, threshold):
  result_final_dict = dict()
  
  #loop từng nghị định
  for number, content in dictionary.items():
    result_dict = {}
    # number : {
    #   content
    # }

    # content có
    # key: sub_dict
    for key, sub_dict in content.items():
      if key == "name" and sub_dict != "":
         result_dict[key] = sub_dict
         continue
      if key =="name" and sub_dict == "":
         continue
      #Trường hợp điều luật không có khoản
      if type(sub_dict) != dict:    
        if key != "name" and sub_dict == "":
            key = key.split(". ")
            result_dict[key[0]] = key[1]
         #nếu câu này > 20 từ
        elif len(sub_dict.split()) >= 20:
            result_dict[key] = sub_dict
        continue

      #Trường hợp điều luật có khoản
      '''
      key: 
      subdict 
      {
        "Dieu 1": abc,
      }
      '''
      total_length = sum(len(value.split()) for value in sub_dict.values())
      if total_length <= threshold:
          result_dict[key] = sub_dict
      else:
        current_length = 0
        temp_dict = {}
        sub_dict_index = 1      
              # '''
              # key: 
              # subdict 
              # {
              #   "Khoan 1": abc,
              #   ...
              #   sub_key: value
              # }
              # '''   
        for sub_key, value in sub_dict.items():
          current_length += len(value.split())
          if current_length <= threshold:
              temp_dict[sub_key] = value
          else:
              name = rename(key, sub_dict_index)
              result_dict[name] = temp_dict  
              temp_dict = {sub_key: value}
              current_length = len(value.split())
              current_lengh = 0
              sub_dict_index += 1
          name = rename(key, sub_dict_index)
          result_dict[name] = temp_dict
    result_final_dict[number] = result_dict
  return result_final_dict

threshold = 1000
result = split_nested_dict_by_length(json_dict, threshold)

In [22]:
len(result)

1857

In [40]:
with open('data_fragment.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

## Data fragment

In [25]:
with open('data_fragment.json', 'r', encoding='utf-8') as f:
  json_dict_frag = json.loads(f.read())

In [26]:
def rename(title, index):
  number = title.split()[1]
  number_change = number[:-1] + f"_{index}."
  title = title.replace(number, number_change)
  return title

def split_nested_dict_by_length(dictionary, threshold):
  result_final_dict = dict()
  
  #loop từng nghị định
  for number, content in dictionary.items():
    result_dict = {}
    # number : {
    #   content
    # }

    # content có
    # key: sub_dict
    for key, sub_dict in content.items():
      if key == "name" and sub_dict != "":
         result_dict[key] = sub_dict
         continue
      if key =="name" and sub_dict == "":
         continue
      #Trường hợp điều luật không có khoản
      if type(sub_dict) != dict:    
        if key != "name" and sub_dict == "":
            key = key.split(". ")
            result_dict[key[0]] = key[1]
         #nếu câu này > 20 từ
        elif len(sub_dict.split()) >= 20:
            result_dict[key] = sub_dict
        continue

      #Trường hợp điều luật có khoản
      '''
      key: 
      subdict 
      {
        "Dieu 1": abc,
      }
      '''
      total_length = sum(len(value.split()) for value in sub_dict.values())
      if total_length <= threshold:
          result_dict[key] = sub_dict
      else:
        current_length = 0
        temp_dict = {}
        sub_dict_index = 1      
              # '''
              # key: 
              # subdict 
              # {
              #   "Khoan 1": abc,
              #   ...
              #   sub_key: value
              # }
              # '''   
        for sub_key, value in sub_dict.items():
          current_length += len(value.split())
          if current_length <= threshold:
              temp_dict[sub_key] = value
          else:
              name = rename(key, sub_dict_index)
              result_dict[name] = temp_dict  
              temp_dict = {sub_key: value}
              current_length = len(value.split())
              current_lengh = 0
              sub_dict_index += 1
          name = rename(key, sub_dict_index)
          result_dict[name] = temp_dict
    result_final_dict[number] = result_dict
  return result_final_dict

threshold = 200
result = split_nested_dict_by_length(json_dict_frag, threshold)

In [27]:
with open('temp.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [47]:
#remove nghi luan chi co ten

A = dict()
for number, content in json_dict_frag.items():
    if len(content) != 1:
        A[number] = content
json_dict_frag = A

In [49]:
with open('final.json', 'w', encoding='utf-8') as f:
    json.dump(json_dict_frag, f, ensure_ascii=False, indent=4)

## Dat cau hoi

In [1]:
import json
with open('temp.json', 'r', encoding='utf-8') as f:
  data_final = json.loads(f.read())

In [2]:
from gpt_api import scrape_answer
from time import sleep

In [4]:
def prettifier(content_dict : dict):
    noi_dung = ""
    for khoan, noi_dung_khoan in content_dict.items():
        noi_dung += f"{khoan}: {noi_dung_khoan}"
    return noi_dung


In [5]:
with open('question/question.json', 'r', encoding='utf-8') as f:
  question_json = json.loads(f.read())

In [34]:
len(question_json)

250

In [57]:
sum(len(item) for item in question_json.values())

1046

In [33]:
question = "Hồng hãy giúp tôi sinh ra 10 câu hỏi dựa vào thông tin trên"

count = 0
for number, content in data_final.items():
    if number in question_json:
        continue
    count += 1
    if count == 24:
        break
    question_json[number] = dict()

    name = ""
    info = ""
    for index, (dieu, noi_dung_dieu) in enumerate(content.items()):
        if index == 6:
            break
        #Lay ten nghi dinh
        if dieu == "name":
            #number = "_".join(number.split("/"))
            name = noi_dung_dieu
            print(name)
            continue 
        #Neu dieu luat khong co khoan
        if type(noi_dung_dieu) != dict:
            info = f"{name}. {dieu}: {noi_dung_dieu}"
            try:
                question_json[number][dieu] = scrape_answer(info, question).split("\n")
                with open('question/question.json', 'w', encoding='utf-8') as f:
                    json.dump(question_json, f, ensure_ascii=False, indent=4)
            except:
                sleep(20)
                question_json[number][dieu] = scrape_answer(info, question).split("\n")
                with open('question/question.json', 'w', encoding='utf-8') as f:
                    json.dump(question_json, f, ensure_ascii=False, indent=4)
                continue
        else: #Neu dieu luat co khoan (noi_dung_dieu)
            noi_dung_dieu = prettifier(noi_dung_dieu)
            info = f"{name}. {dieu}: {noi_dung_dieu}"
            try:
                question_json[number][dieu] = scrape_answer(info, question).split("\n")
                with open('question/question.json', 'w', encoding='utf-8') as f:
                    json.dump(question_json, f, ensure_ascii=False, indent=4)
            except:
                sleep(20)
                question_json[number][dieu] = scrape_answer(info, question).split("\n")
                with open('question/question.json', 'w', encoding='utf-8') as f:
                    json.dump(question_json, f, ensure_ascii=False, indent=4)
                continue

Nghị định 150/2020/NĐ-CP chuyển đơn vị sự nghiệp công lập thành công ty cổ phần
Nghị định 100/2019/NĐ-CP xử phạt vi phạm hành chính lĩnh vực giao thông đường bộ và đường sắt
Luật Hộ tịch 2014 số 60/2014/QH13
Luật Đường sắt 2017 số 06/2017/QH14
Nghị quyết 201/NQ-HĐND 2019 thông qua bảng giá đất tỉnh Gia Lai
Nghị định 96/2020/NĐ-CP xử phạt vi phạm hành chính trong quản lý bảo vệ biên giới quốc gia
Pháp lệnh 02/2020/UBTVQH14 ưu đãi người có công với Cách mạng
Nghị định 49/2016/NĐ-CP sửa đổi 109/2013/NĐ-CP xử phạt hành chính quản lý giá phí lệ phí hóa đơn
Nghị định 95/2018/NĐ-CP đăng ký giao dịch công cụ nợ của Chính phủ trên thị trường chứng khoán
Quyết định 12/2020/QĐ-UBND ban hành Quy định về Bảng giá đất tỉnh Hà Nam 2020 2024
Nghị định 111/2022/NĐ-CP hợp đồng một số loại công việc trong cơ quan hành chính
Nghị định 71/2022/NĐ-CP sửa đổi Nghị định 06/2016/NĐ-CP sử dụng dịch vụ phát thanh
Thông tư liên tịch 01/2016/TTLT-TANDTC-VKSNDTC-BTP hướng dẫn thi hành Luật hôn nhân gia đình
Thông t

In [ ]:
question_json['88/2020/NĐ-CP']["Điều 2_2. Đối tượng áp dụng"]

In [ ]:
data_final['54/2019/QH14']["Điều 4_2_4. Giải thích từ ngữ"]

In [53]:
with open('question/question.json', 'r', encoding='utf-8') as f:
  question_json = json.loads(f.read())

In [55]:
len(question_json)

250

In [54]:
count = 0
for number, content in question_json.items():
    for dieu, list_cau_hoi in content.items():
        if len(list_cau_hoi) != 10:
            print(number)
            print(dieu)

## Answers

In [1]:
import json

with open('question/question.json', 'r', encoding='utf-8') as f:
  question_json = json.loads(f.read())

with open('temp.json', 'r', encoding='utf-8') as f:
  data_final = json.loads(f.read())

with open('answer/answer.json', 'r', encoding='utf-8') as f:
  answer_json = json.loads(f.read())

In [2]:
# answer_json = {}
def prettifier(content_dict : dict):
    noi_dung = ""
    for khoan, noi_dung_khoan in content_dict.items():
        noi_dung += f"{khoan}: {noi_dung_khoan}"
    return noi_dung

In [ ]:
from answer.api import scrape_answer
from time import sleep

count = 0
for checkpoint, (number, content) in enumerate(question_json.items()):
    if number in answer_json:
        continue
    if count == 3:
        break
    count += 1
    answer_json[number] = {}
    index = 1
    for dieu, list_cau_hoi in content.items():
        if type(data_final[number][dieu]) == dict:
            info = prettifier(data_final[number][dieu])
        else:
            info = data_final[number][dieu]
        for cau_hoi in list_cau_hoi:
            cau_hoi = cau_hoi.split(". ")[1]
            try:
                answer = scrape_answer(info, cau_hoi)
                answer_json[number][index] = {
                    'Question': cau_hoi,
                    'Context': info,
                    'Answer': answer
                }
                index += 1
                with open('answer/answer.json', 'w', encoding='utf-8') as f:
                    json.dump(answer_json, f, ensure_ascii=False, indent=4)
            except:
                sleep(20)
                answer = scrape_answer(info, cau_hoi)
                answer_json[number][index] = {
                    'Question': cau_hoi,
                    'Context': info,
                    'Answer': answer
                }
                with open('answer/answer.json', 'w', encoding='utf-8') as f:
                    json.dump(answer_json, f, ensure_ascii=False, indent=4)
                index += 1

In [3]:
print(len(question_json))
print(len(answer_json))

print(sum(len(value) for value in answer_json.values()))

250
53
1579
